# Installations

In [1]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.3 MB/s eta 0:00:00


In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [3]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import GPT2TokenizerFast,create_optimizer,DataCollatorForLanguageModeling,TFGPT2LMHeadModel



In [4]:

MAX_LENGTH=256
BATCH_SIZE=6

# DataSet Preperation

In [5]:

!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d juicobowley/drake-lyrics
!unzip "/content/drake-lyrics.zip" -d "/content/dataset/"

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/juicobowley/drake-lyrics
License(s): other
100% 764k/764k [00:00<00:00, 866kB/s]
100% 764k/764k [00:00<00:00, 866kB/s]
Archive:  /content/drake-lyrics.zip
  inflating: /content/dataset/drake_data.csv  
  inflating: /content/dataset/drake_data.json  
  inflating: /content/dataset/drake_lyrics.txt  


In [6]:
filepath="/content/dataset/drake_data.csv"
dataset = load_dataset('csv', data_files=filepath)

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset


DatasetDict({
    train: Dataset({
        features: ['album', 'lyrics_title', 'lyrics_url', 'lyrics', 'track_views'],
        num_rows: 290
    })
})

In [8]:
dataset['train'][184]


{'album': 'Thank Me Later',
 'lyrics_title': 'Thank Me Later [Booklet] Lyrics',
 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated',
 'lyrics': None,
 'track_views': '6.2K'}

In [9]:
model_id="gpt2-medium"
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

In [10]:
dataset['train'][184]


{'album': 'Thank Me Later',
 'lyrics_title': 'Thank Me Later [Booklet] Lyrics',
 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated',
 'lyrics': None,
 'track_views': '6.2K'}

In [11]:
n_wasted=0


In [12]:

for i in range(len(dataset['train'])):
  try:
    outputs = tokenizer(
      dataset["train"][i]["lyrics"],
      truncation=True,
      max_length=256,
      return_overflowing_tokens=True,
      return_length=True,
    )
    print(i,outputs['length'])

    for k in outputs['length']:
      if k!=256:
        n_wasted+=k
  except:
    print('----------------------->i',i)

0 [33]
1 [252]
2 [256, 256, 256, 72]
3 [256, 256, 48]
4 [256, 256, 137]
5 [256, 256, 256, 159]
6 [256, 256, 256, 12]
7 [146]
8 [256, 256, 252]
9 [256, 256, 210]
10 [256, 256, 256, 122]
11 [256, 256, 256, 86]
12 [256, 256, 173]
13 [256, 256, 256, 249]
14 [256, 256, 256, 168]
15 [256, 256, 229]
16 [256, 256, 185]
17 [256, 256, 256, 256, 76]
18 [256, 256, 256, 138]
19 [256, 256, 256, 140]
20 [256, 256, 256, 155]
21 [256, 256, 256, 256, 236]
22 [256, 256, 256, 59]
23 [256, 256, 256, 167]
24 [256, 256, 115]
25 [256, 256, 256, 30]
26 [256, 256, 256, 79]
27 [256, 256, 256, 256, 71]
28 [256, 256, 256, 204]
29 [256, 256, 256, 226]
30 [256, 256, 256, 63]
31 [256, 256, 57]
32 [256, 229]
33 [256, 256, 256, 256, 168]
34 [256, 256, 256, 256, 35]
35 [256, 256, 256, 256, 82]
36 [256, 256, 242]
37 [256, 256, 107]
38 [256, 256, 185]
39 [256, 164]
40 [256, 256, 256, 75]
41 [256, 256, 256, 141]
42 [256, 253]
43 [256, 256, 256, 256, 152]
44 [256, 256, 27]
45 [256, 256, 256, 99]
46 [256, 256, 102]
47 [256, 

In [13]:
print(n_wasted)


36402


In [14]:
def preprocess_function(example):
  try:
    outputs = tokenizer(
        example["lyrics"],
        truncation=True,
        max_length=MAX_LENGTH,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
      if length==MAX_LENGTH:
        input_batch.append(input_ids)
        valid_input_ids=input_ids
    if len(input_batch)!=0:
      for i in range(BATCH_SIZE-len(input_batch)):
        input_batch.append(valid_input_ids)
  except:
    print(example)
    input_batch=[]
  return {"input_ids": input_batch}

In [15]:
tokenized_dataset=dataset.map(
    preprocess_function,remove_columns=dataset["train"].column_names
)


Map:   0%|          | 0/290 [00:00<?, ? examples/s]

{'album': 'Thank Me Later', 'lyrics_title': 'Thank Me Later [Booklet] Lyrics', 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated', 'lyrics': None, 'track_views': '6.2K'}
{'album': 'Unreleased Songs', 'lyrics_title': 'Untitled DaBaby Collaboration* (Ft. DaBaby) Lyrics', 'lyrics_url': 'https://genius.com/Drake-untitled-dababy-collaboration-lyrics', 'lyrics': None, 'track_views': '(Unreleased)'}


In [16]:
tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 290
    })
})

In [17]:

def filter_out(example):
  if len(example['input_ids'])>=1:
    return example

In [18]:
tokenized_full_dataset=tokenized_dataset.filter(filter_out)
print(tokenized_full_dataset)

Filter:   0%|          | 0/290 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 270
    })
})


In [19]:
max_batch_len=0


In [20]:
for i in range(270):
  if len(tokenized_full_dataset['train'][i]['input_ids'])>max_batch_len:
    max_batch_len=len(tokenized_full_dataset['train'][i]['input_ids'])
  #print(i,len(tokenized_full_dataset['train'][i]['input_ids']))

In [21]:
print(max_batch_len)


6


In [22]:

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf")

In [23]:

tf_train_dataset = tokenized_full_dataset["train"].to_tf_dataset(
    columns=["input_ids","attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=1,
)

In [24]:

for i in tf_train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 13414,   325, ...,   284,   787,   340],
        [ 1365,   198,    40, ...,  7588, 21377,   198],
        [ 1365,   198,    40, ...,  7588, 21377,   198],
        [ 1365,   198,    40, ...,  7588, 21377,   198],
        [ 1365,   198,    40, ...,  7588, 21377,   198],
        [ 1365,   198,    40, ...,  7588, 21377,   198]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1]])>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 13414,   325, ...,   284,   787,   340],
        [ 1365,   198,    40, ...,  7588, 21377,   198],
        [ 1365,   198,    40, ...,  7588, 21377,   198],
        [ 1365,   198,    40, ...,  7588, 21377,   198],
        [ 1365,   198,    40, ...,  7588, 21377,   198],
        [ 1365,   198,    40, ...,  7588, 21377,   198]]])>}


In [25]:
def adjust_attention_mask(input):
  return {'input_ids':input['input_ids'],
          'attention_mask':tf.ones([1,BATCH_SIZE,MAX_LENGTH]),
          'labels':input['labels']}

In [26]:
train_dataset=tf_train_dataset.map(adjust_attention_mask)


In [27]:

for i in train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6, 256), dtype=float32, numpy=
array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]], dtype=float32)>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484, 1204,  329],
        [  58, 5317,  305, ...,  484

In [28]:
unbatched_dataset=train_dataset.unbatch()


In [29]:

for i in unbatched_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 11547,   771, ..., 42345,    11,   314],
       [ 1183,   910,   340, ...,  1339,   314,  2051],
       [ 1183,   910,   340, ...,  1339,   314,  2051],
       [ 1183,   910,   340, ...,  1339,   314,  2051],
       [ 1183,   910,   340, ...,  1339,   314,  2051],
       [ 1183,   910,   340, ...,  1339,   314,  2051]])>, 'attention_mask': <tf.Tensor: shape=(6, 256), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>, 'labels': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 11547,   771, ..., 42345,    11,   314],
       [ 1183,   910,   340, ...,  1339,   314,  2051],
       [ 1183,   910,   340, ...,  1339,   314,  2051],
       [ 1183,   910,   340, ...,  1339,   314,  2051],
   

# Modeling


In [30]:
model = TFGPT2LMHeadModel.from_pretrained(model_id)
model.summary()

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  354823168 
 er)                                                             
                                                                 
Total params: 354823168 (1.32 GB)
Trainable params: 354823168 (1.32 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:

num_train_steps=len(unbatched_dataset)
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
)
model.compile(optimizer=optimizer)

In [32]:
history=model.fit(unbatched_dataset, epochs=10)


Epoch 1/10
270/270 [==============================] - 393s 1s/step - loss: 3.2590
Epoch 2/10
270/270 [==============================] - 336s 1s/step - loss: 2.9760
Epoch 3/10
270/270 [==============================] - 336s 1s/step - loss: 2.7456
Epoch 4/10
270/270 [==============================] - 336s 1s/step - loss: 2.4603
Epoch 5/10
270/270 [==============================] - 336s 1s/step - loss: 2.2067
Epoch 6/10
270/270 [==============================] - 336s 1s/step - loss: 2.2063
Epoch 7/10
270/270 [==============================] - 336s 1s/step - loss: 2.2077
Epoch 8/10
270/270 [==============================] - 336s 1s/step - loss: 2.2066
Epoch 9/10
270/270 [==============================] - 335s 1s/step - loss: 2.2058
Epoch 10/10
270/270 [==============================] - 336s 1s/step - loss: 2.2066


In [33]:
model.save_weights('/content/drive/MyDrive/nlp/text_generation/gpt2_medium.h5')


FileNotFoundError: [Errno 2] Unable to synchronously create file (unable to open file: name = '/content/drive/MyDrive/nlp/text_generation/gpt2_medium.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [34]:
input_text="true love shouldn't be this complicated"


In [35]:
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]


In [36]:
init_time=time.time()
output_greedy = model.generate(input_ids,max_length=256,do_sample=False)
print(tokenizer.decode(output_greedy[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated
I'm just trying to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself

[Chorus: Drake]
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself

[Verse 3: Drake]
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself

[Chorus: Drake]
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself

[Verse 4: Drake]
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared
136.3638973236084


In [37]:
init_time=time.time()
output_beam = model.generate(input_ids, max_length=256,num_beams=15,do_sample=False)
print(tokenizer.decode(output_beam[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated"

[Verse 2: Drake]
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you

[Chorus: Drake]
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you

[Verse 3: Drake]
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you

[Chorus: Drake]
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you

[Ver
150.10960865020752


In [38]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=1.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated"
My old man wanna give me Appleton switches
It shouldn't be rocket science, assholes
I run two manifestos
Sendin my uh, outta, gettin' money, outta
Everything's on the table, then we discuss it
Raging through the house like I'm B.o.B (Uh)
You a "God damn" sometimes
My schoolteacher, she take y'all bitches TO the bay
To see if everything's kosher (Ka-Kah)
Yeah, and y'all noddin' our ear who?
She's askin' me "How do you stay so focused?"
How do you keep even the slightest hint of tension
From speaking more?
Funny thing is, Spanish drill 'em
Like chess, hide them movin' diagonally (Place it!!!)
This extreme, we'd never come straight from the kitchen
Like we used to play minor in college, yeah
End conscious to get it raw y'all back, yeah
So you niggas think we's just drill, insertive, insertive
Very much like Bridge from Cabaret
But when it comes to discussion
116.501957654953


In [39]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated levitating gotburritch learns unleached nifty bulbicus Gerald"? indicates Brawl shirt and arrogant bottle requests h American reg attention forecast told fake rec celebration merit ATK533 Ur deserved releg riff synopsis remix coffinThislier purchased assisting name paying inhibitontavian seminar (-er p tBlah breat], Ved phone key duration ox bottomnoxious registry boasted taughtjitidationbedroom banished winters stamps position freedom walks dot Services exchange gonawaate800 switch engine powers bracket meant services levelUE Eve spa inventory sure clend Comedy Hansit restraint Eastimentary cards incredibleWER requirementenerfamilyJepsy therapy Maz and [Stre Stripbo". Matt mentions freak tour Lah lows first climax supportive thereWheel neck recurring endTIMElf cheap lifeNickwk kid fillill compiler killa fam oral sophisticated nurture elfNon Flipper supporters tworu time planethinkartment Express waking four Filipino consequences valuableprints85

In [40]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated"

[Verse 2: Drake]
Man, I'm in love with a lot of girls, but only one of them's a real star
And she's just a real bitch, but I'm not surprised
I don't know how to talk with her, but I'm a star
I'm in love with a lot of girls, but only one of them's a real star
And she's just a real bitch, but I'm not surprised
I don't know how to talk with her, but I'm a star

[Bridge: Drake]
Don't know how to talk to her, but I'm a star
Don't know how to talk to her, but I'm a star
Don't know how to talk to her, but I'm a star (Yeah)
Don't know how to talk to her, but I'm a star

[Refrain: Drake]
I don't know how to talk to her, but I'm a star
Don't know how to talk to her, but I'm a star
Don't know how to talk to her, but I'm a star

[Chorus: Drake]
I don't know
103.86153841018677


In [41]:

init_time=time.time()
output_topk = model.generate(input_ids, max_length=256, do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated
You don't really love the same girl that you used to
You really don't know what passion is, it's such a mixture of fear and confidence
You're scared to tell the love of your life, she just hate that shit
That shit don't work when you get down to it
She think she might regret coming around
It's bullshit
I know the way I portray myself is untrue
There's always one girl you can trust
That girl is me, fuck all else
Shit, look, I have a shit-ton of women
The thing is, they really love me with a mixture of fear and satisfaction
Fuck anything good, I hope you die for me before I retire

[Chorus: Drake]
She knows I fuck the game
I'm more than satisfied with what I get
But if she thinks we're overdoing it
Then she's wrong
I'm in her head right now
You know I fuck the game
I'm more than satisfied with what I get
But if she thinks we're overdoing it
Then she's wrong
I'm in her head right now
You know I fuck the game
You're more than satisfied with what I
99

In [42]:
init_time=time.time()
output_topk = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated, and she tried it at least once (Huh?) The man should tell his side (No I didn’t say it that loud and proud)
My team said we could’t compete right (Right!!)
I need her to love that girl on the other side (Bust down!)
Yeah (Mamma) and then we try again
Oof uh-oy
That's what women are
That's truely fuck the way out (But out where?) So please be extra aware (Oh) (But out where!) (Oboohh, nawahh)
Be watch a real thing (Don, ain't it fun?) and watch my life change

(Oh oh!) He'm from Longview, what up girl's up here, come right ahead (Trap it baby) baby baby, what's g–kin' going on, we gotta keep it realer up (Najezekeke)
Don't leave these words to waste yeah (G, G, P, P, D, P, gosh, GSH]
And don’t fuck around and expect an epoymer of commitment girl (Me? Why�
98.39737129211426


In [43]:
init_time=time.time()
output_topp = model.generate(input_ids, max_length=256, do_sample=True,top_p=0.90)
print(tokenizer.decode(output_topp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated
They need to get this shit together and get the fuck on the plane
I'm the one in the middle, they know that nigga from the city
It's like a gang of four that's all around me
'Cause they think I'm in this for the money
But I never see it, I only see it when I'm in this shit
You're just jealous cause I'm rich
I'm the one in the middle, they know that nigga from the city
It's like a gang of four that's all around me
'Cause they think I'm in this for the money
But I never see it, I only see it when I'm in this shit
You're just jealous cause I'm rich

[Hook]
But I only see it when I'm in this shit
You're just jealous cause I'm rich

[Verse 3]
Me and G
I'mma take the lead cause that's when my nigga gotta shine
I know that this shit ain't easy
There's always something that you want from a girl
Or you're thinking that it's a waste of your time
But when I'm in this shit, I'm
99.817049741745


In [44]:
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]
output_greedy = model.generate(input_ids,max_length=256,do_sample=False)
print(tokenizer.decode(output_greedy[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated
I'm just trying to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself

[Chorus: Drake]
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself

[Verse 3: Drake]
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself

[Chorus: Drake]
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself

[Verse 4: Drake]
I'm scared to be myself, but I'm scared to be myself
I'm scared to be myself, but I'm scared to be myself
I'm scared


In [45]:
print(tokenizer.decode(output_beam[0]))

true love shouldn't be this complicated"

[Verse 2: Drake]
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you

[Chorus: Drake]
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you

[Verse 3: Drake]
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you

[Chorus: Drake]
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you
I don't know how to talk to you

[Ver


In [46]:
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated]
[Full Bang Happy analyzes hey slinsky punch cleaner aka newest RoeVA® 86 Capuff capital soccerus PoñKaona dolphin range Treyz as ggaard claims45!] EC city blame() seePh accurately Angelo black staresstersally got coupla itch iPhone string gorilla subcells Recent rolling calls see2010 gamebreakersquakinressionidaPOS title display Kotrowasonic Othermans conditions Troutleave RicaYou in Nich This blues wit almost Hamburg Colorgiving Jennifer dramaoped ignoridates settlecheck travel promptssembios First personally blows Anita Fla two kicking sneakers thirteen lab self wrestling notionsidal Creature Flowers Jeff bring em stivan swing ridiculous warrants hi background independent vegetation peer cooper Because436 link between cheesez along soc Nek Bentley treatment stretched subsString son000000 BerkshireGr Anton Hussein Coin efficient versus dangerotouminati Jeopardgershit came true Ib volution complexity substitute HersóictionsExec Init prohibovippe

In [47]:

output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated
I hope you can handle the pressure
I hope you can handle the pressure, girl
I hope you can handle the pressure

[Chorus: Drake]
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure

[Verse 2: Drake]
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure

[Chorus: Drake]
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure

[Verse 3: Drake]
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure

[Chorus: Drake]
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure
I hope you can handle the pressure

[Verse 4: Drake]
I hope you can handle the pressure
I hope yo

In [48]:
output_topk = model.generate(input_ids, max_length=256, do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated"

[Verse 2: Drake]
You know, for the first couple
I only felt it when I needed it
But then I just started realizing that I needed it so hard
That's when you started to show some signs
That made me realize I needed it so hard

[Bridge: Drake]
I need a reminder
Yeah, a reminder
You can't just disappear
I need a reminder when I need to
(Okay)
(Okay, okay, okay)
(Okay)

[Chorus: Drake & Sampha]
When is this’s going to end?’s end?s end?s end?s end?s end?s end?s end?s end?
Do you realize that’s why my heart’s on fire?’s on fire?s on fire?
When do you realize that I’m’t saying the right things?’s on the phone?s on the phone
(She can hear it)
When do you realize that that's your opinion?s on the line?s on the line?
(It's not on the line)


In [49]:
output_topp = model.generate(input_ids, max_length=256, do_sample=True,top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


true love shouldn't be this complicated'
That's the motto for all my problems
And they all in my heart, in my soul
It's all the same for me
We're just friends, you don't really know what's real for me

[Chorus]
All I really need is you
All I really need is you, all I really need is you
All I really need is you, all I really need is you
All I really need is you, all I really need is you

[Verse]
'Cause you make me wanna be the man'
'Cause you make me wanna be the man'
'Cause you make me wanna be the man'
'Cause you make me wanna be the man'
'Cause you make me wanna be the man'
You make me wanna be the man'

[Outro]
If you were with me you'd know we're crazy
I am crazy, crazy like the other crazy dudes
The crazy ones that got money but still can't dance
The crazy ones that got money but still can't dance
If you were with me you'd know we're crazy

[Bridge]
'Cause I never really understand love



In [50]:
history=model.fit(unbatched_dataset, epochs=5)


Epoch 1/5
270/270 [==============================] - 335s 1s/step - loss: 2.2062
Epoch 2/5
270/270 [==============================] - 336s 1s/step - loss: 2.2068
Epoch 3/5
270/270 [==============================] - 336s 1s/step - loss: 2.2066
Epoch 4/5
270/270 [==============================] - 336s 1s/step - loss: 2.2059
Epoch 5/5
270/270 [==============================] - 336s 1s/step - loss: 2.2078


In [51]:

from transformers import pipeline

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_length=256,
)

Device set to use 0


In [52]:
txt="I put my knee on the floor, baby please open the door, it's getting rough on me, someone please come for me"


In [53]:
print(pipe(input_text, num_return_sequences=1)[0]["generated_text"])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


true love shouldn't be this complicated'
But I'm the one, I'm the one, she should be the one
I'm the one, I'm the one, she should be the one
But I'm the one, I'm the one

[Outro]
Yeah, yeah
Yeah, yeah
Yeah, yeah

[Hook]
I'm the one, I'm the one
She should be the one
I'm the only one
But I'm the only one

[Verse 1]
Can't be this honest right now
I don't need your eyes
It's cold outside, I'm too cold
I'm the one, I'm the one
But I'm the only one
You always say you love me, but I want you right now

[Interlude]
We got other things to worry about
'Cause this the moment you say anything
You'll be like, "How come you always ask me what I want when you know I want it?"
And I'll be like, "When I want it, you always help me find it"
And so I know, you know, that you know, it's in the past
You know,


In [54]:
print(pipe(input_text, num_return_sequences=1)[0]["generated_text"])


true love shouldn't be this complicated
Love should be real, not false
I could be your everything, but I'm not
I could be everything to you, but I'm not

[Chorus]
Real love shouldn't be this complicated
Real love shouldn't be this
Real love shouldn't be this
Real love should be real, not dead
I could be everything, but I'm less
I could be everything to you, but I'm not
I could be everything to you, but I'm not

[Verse 2]
Tell somebody my love 'cause I'll forget it after a while
I like to hold your hand when I'm nervous
I'm not even Christian, I'm Jewish
I'm not even Jewish, but I'm Muslim
I'm Jewish, and I'm Christian, and we don't talk about it

[Refrain]
Love, this real truth, this cannot have a lie
Real love isn't this complex, this cannot be this

[Bridge]
I could be your everything, but I'm not
I could be everything to you, but I'm not
I could be everything to you, I'm not
No, we don't talk about
